In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
import matplotlib.backends.backend_pdf

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

%matplotlib inline

dir_data='/Users/ahenny/'
dir1='/Volumes/LaCie/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/day/'

In [ ]:
yrs=np.arange(2001,2021,1)

for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir_data+'holding/'+'gpcp3_precip_'+str(year)+'.nc')
    precip=ds['precip']
    
    defined1=xr.ones_like(precip).sum(dim='time')
    defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time')
    
    print(defined.mean().values.tolist())
    
    if i==0:
        max_concat=defined1
        present_concat=defined
    else:
        max_concat=xr.concat([max_concat,defined1],dim='time')
        present_concat=xr.concat([present_concat,defined],dim='time')

In [ ]:
max_all=max_concat.sum(dim='time')
present_all=present_concat.sum(dim='time')
ratio_all=present_all/max_all

In [ ]:
yrs=np.arange(2001,2021,1)

ds=xr.open_dataset(dir_data+'gpcp3_threshold_final_98_all.nc')
#ds=xr.open_dataset(dir_data+'gpcp3_threshold_r99p.nc')
lats_sel=[x for x in ds.lat.values if -60<=x<=70]
threshold=ds['threshold'].sel(lat=lats_sel)
ratio_all=ratio_all.sel(lat=lats_sel)

if 1==1:
    total_list_land=[]
    freq_list_land=[]
    intensity_list_land=[]
    
    total_list_ocean=[]
    freq_list_ocean=[]
    intensity_list_ocean=[]
    
    total_ocean_monthly=np.zeros((12,len(yrs)))
    freq_ocean_monthly=np.zeros((12,len(yrs)))
    intensity_ocean_monthly=np.zeros((12,len(yrs)))
    
    total_land_monthly=np.zeros((12,len(yrs)))
    freq_land_monthly=np.zeros((12,len(yrs)))
    intensity_land_monthly=np.zeros((12,len(yrs)))

for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    factor=1.
        
    ds=xr.open_dataset(dir_data+'holding/'+'gpcp3_precip_'+str(year)+'.nc')
    precip=ds['precip'].sel(lat=lats_sel)
    weights=np.cos(np.deg2rad(precip.lat))

    if i==0:#land mask option
        ds=xr.open_dataset('/Users/ahenny/Downloads/data.nc')
        t2m=ds['t2m'][0,:,:]
        land_mask=t2m/t2m
        land_mask=land_mask.fillna(0)
        
        lon_east=[x for x in land_mask.longitude.values if x>=180.]
        lon_west=[x for x in land_mask.longitude.values if x<180.]
        l_east=land_mask.sel(longitude=lon_east)
        l_west=land_mask.sel(longitude=lon_west)
        l_east['longitude']=[x-360. for x in lon_east]
        land_mask=xr.concat([l_east,l_west],dim='longitude')
        
        dk=xr.Dataset()
        dk['land']=(('lat','lon'),land_mask.values)
        dk.coords['lat']=land_mask.latitude.values
        dk.coords['lon']=land_mask.longitude.values
        
        land_mask=dk['land']
        land_mask_interp=land_mask.interp_like(precip[0,:,:])
        
    defined1=xr.ones_like(precip).sum(dim='time').where(land_mask_interp>=0.5)
    defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time').where(land_mask_interp>=0.5)
    ratio_land=defined/defined1
    valid_area_land=xr.ones_like(precip).where(ratio_land>=0.9).fillna(0).where(land_mask_interp>=0.5).weighted(weights).mean().values.tolist()
    
    defined1=xr.ones_like(precip).sum(dim='time').where(land_mask_interp<0.5)
    defined=xr.ones_like(precip).where(precip.round(2)>=0).sum(dim='time').where(land_mask_interp<0.5)
    ratio_ocean=defined/defined1
    valid_area_ocean=xr.ones_like(precip).where(ratio_ocean>=0.9).fillna(0).where(land_mask_interp<0.5).weighted(weights).mean().values.tolist()
    
    print((valid_area_land,valid_area_ocean))

    if 1==1:
        threshold_new=xr.zeros_like(precip)
        for k in range(threshold_new.time.size):
            date=pd.to_datetime(threshold_new.time.values[k])
            if date.month==2 and date.day==29:
                threshold_new1=threshold.sel(time=dt.datetime(2001,2,28))
                threshold_new2=threshold.sel(time=dt.datetime(2001,3,1))
                threshold_new3=(threshold_new1+threshold_new1)/2.
            else:
                threshold_new3=threshold.sel(time=dt.datetime(2001,date.month,date.day))

            threshold_new[k,:,:]=threshold_new3.values
    if 1==0:
        threshold_new=threshold
        
    precip_extreme=precip.where(precip>=1.)
    #precip_extreme=precip_extreme.where(precip_extreme>=threshold_new)
    
    #OCEAN
    
    ones=precip_extreme/precip_extreme
    freq=ones.sum(dim='time',skipna=True).fillna(0).where(land_mask_interp<0.5).where(ratio_all>=0.9).where(ratio_ocean>=0.9)/ratio_ocean
    extreme_sum=precip_extreme.sum(dim='time',skipna=True).fillna(0).where(land_mask_interp<0.5).where(ratio_all>=0.9).where(ratio_ocean>=0.9)/ratio_ocean
    #freq=ones.sum(dim='time',skipna=True).fillna(0).where(land_mask_interp<0.5)
    #extreme_sum=precip_extreme.sum(dim='time',skipna=True).fillna(0).where(land_mask_interp<0.5)
    
    weights=np.cos(np.deg2rad(extreme_sum.lat))
    total_weighted=extreme_sum.weighted(weights)
    freq_weighted=freq.weighted(weights)
    weighted_1=precip_extreme.where(land_mask_interp<0.5).where(ratio_all>=0.9).where(ratio_ocean>=0.9).weighted(weights)
    #weighted_1=precip_extreme.where(land_mask_interp<0.5).weighted(weights)
    intensity=weighted_1.mean(skipna=True).values.tolist()
    
    if valid_area_ocean>=0.9:
    #if 1==1:
        total_list_ocean.append(total_weighted.mean(dim=('lat','lon'),skipna=True).values.tolist()*factor)
        freq_list_ocean.append(freq_weighted.mean(dim=('lat','lon'),skipna=True).values.tolist()*factor)
        intensity_list_ocean.append(intensity)
    else:
        total_list_ocean.append(np.nan)
        freq_list_ocean.append(np.nan)
        intensity_list_ocean.append(np.nan)

    for j in range(12):
        dates=[x for x in ones.time.values if pd.to_datetime(x).month==j+1]
        precip_sel=precip.sel(time=dates)
        defined1=xr.ones_like(precip_sel).sum(dim='time').where(land_mask_interp<0.5)
        defined=xr.ones_like(precip_sel).where(precip_sel.round(2)>=0).sum(dim='time').where(land_mask_interp<0.5)
        ratio=defined/defined1
        valid_area=xr.ones_like(precip_sel).where(ratio>=0.9).fillna(0).where(land_mask_interp<0.5).weighted(weights).mean().values.tolist()
    
        field=precip_extreme.sel(time=dates)
        extreme_sum=field.sum(dim='time',skipna=True).fillna(0).where(land_mask_interp<0.5).where(ratio_all>=0.9).where(ratio>=0.9)/ratio
        #extreme_sum=field.sum(dim='time',skipna=True).fillna(0).where(land_mask_interp<0.5)
        total_weighted=extreme_sum.weighted(weights)
        total_mean=total_weighted.mean(dim=('lat','lon'),skipna=True).values.tolist()

        field=ones.sel(time=dates)
        ones_sum=field.sum(dim='time',skipna=True).fillna(0).where(land_mask_interp<0.5).where(ratio_all>=0.9).where(ratio>=0.9)/ratio
        #ones_sum=field.sum(dim='time',skipna=True).fillna(0).where(land_mask_interp<0.5)
        freq_weighted=ones_sum.weighted(weights)
        freq_mean=freq_weighted.mean(dim=('lat','lon'),skipna=True).values.tolist()
        
        field=precip_extreme.sel(time=dates)
        intensity_weighted=field.where(land_mask_interp<0.5).where(ratio_all>=0.9).where(ratio>=0.9).weighted(weights)
        #intensity_weighted=field.where(land_mask_interp<0.5).weighted(weights)
        intensity_mean=intensity_weighted.mean(skipna=True).values.tolist()
        
        if valid_area>=0.9:
            total_ocean_monthly[j,i]=total_mean*factor
            freq_ocean_monthly[j,i]=freq_mean*factor
            intensity_ocean_monthly[j,i]=intensity_mean
        else:
            total_ocean_monthly[j,i]=np.nan
            freq_ocean_monthly[j,i]=np.nan
            intensity_ocean_monthly[j,i]=np.nan
    
    #LAND
    
    freq=ones.sum(dim='time',skipna=True).fillna(0).where(land_mask_interp>=0.5).where(ratio_all>=0.9).where(ratio_land>=0.9)/ratio_land
    extreme_sum=precip_extreme.sum(dim='time',skipna=True).fillna(0).where(land_mask_interp>=0.5).where(ratio_all>=0.9).where(ratio_land>=0.9)/ratio_land
    #freq=ones.sum(dim='time',skipna=True).fillna(0).where(land_mask_interp>=0.5)
    #extreme_sum=precip_extreme.sum(dim='time',skipna=True).fillna(0).where(land_mask_interp>=0.5)
    
    weights=np.cos(np.deg2rad(extreme_sum.lat))
    total_weighted=extreme_sum.weighted(weights)
    freq_weighted=freq.weighted(weights)
    weighted_1=precip_extreme.where(land_mask_interp>=0.5).where(ratio_all>=0.9).where(ratio_land>=0.9).weighted(weights)
    #weighted_1=precip_extreme.where(land_mask_interp>=0.5).weighted(weights)
    intensity=weighted_1.mean(skipna=True).values.tolist()
    
    if valid_area_land>=0.9:
    #if 1==1:
        print(total_weighted.mean(dim=('lat','lon'),skipna=True).values.tolist()*factor)
        total_list_land.append(total_weighted.mean(dim=('lat','lon'),skipna=True).values.tolist()*factor)
        freq_list_land.append(freq_weighted.mean(dim=('lat','lon'),skipna=True).values.tolist()*factor)
        intensity_list_land.append(intensity)
    else:
        total_list_land.append(np.nan)
        freq_list_land.append(np.nan)
        intensity_list_land.append(np.nan)

    for j in range(12):
        dates=[x for x in ones.time.values if pd.to_datetime(x).month==j+1]
        precip_sel=precip.sel(time=dates)
        defined1=xr.ones_like(precip_sel).sum(dim='time').where(land_mask_interp>=0.5)
        defined=xr.ones_like(precip_sel).where(precip_sel.round(2)>=0).sum(dim='time').where(land_mask_interp>=0.5)
        ratio=defined/defined1
        valid_area=xr.ones_like(precip_sel).where(ratio>=0.9).fillna(0).where(land_mask_interp>=0.5).weighted(weights).mean().values.tolist()
        
        field=precip_extreme.sel(time=dates)
        extreme_sum=field.sum(dim='time',skipna=True).fillna(0).where(land_mask_interp>=0.5).where(ratio_all>=0.9).where(ratio>=0.9)/ratio
        #extreme_sum=field.sum(dim='time',skipna=True).fillna(0).where(land_mask_interp>=0.5)
        total_weighted=extreme_sum.weighted(weights)
        total_mean=total_weighted.mean(dim=('lat','lon'),skipna=True).values.tolist()

        field=ones.sel(time=dates)
        ones_sum=field.sum(dim='time',skipna=True).fillna(0).where(land_mask_interp>=0.5).where(ratio_all>=0.9).where(ratio>=0.9)/ratio
        #ones_sum=field.sum(dim='time',skipna=True).fillna(0).where(land_mask_interp>=0.5)
        freq_weighted=ones_sum.weighted(weights)
        freq_mean=freq_weighted.mean(dim=('lat','lon'),skipna=True).values.tolist()

        field=precip_extreme.sel(time=dates)
        intensity_weighted=field.where(land_mask_interp>=0.5).where(ratio_all>=0.9).where(ratio>=0.9).weighted(weights)
        #intensity_weighted=field.where(land_mask_interp>=0.5).weighted(weights)
        intensity_mean=intensity_weighted.mean(skipna=True).values.tolist()
        
        if valid_area>=0.9:
            total_land_monthly[j,i]=total_mean*factor
            freq_land_monthly[j,i]=freq_mean*factor
            intensity_land_monthly[j,i]=intensity_mean
        else:
            total_land_monthly[j,i]=np.nan
            freq_land_monthly[j,i]=np.nan
            intensity_land_monthly[j,i]=np.nan

In [ ]:
dk=xr.Dataset()
dk['total_list_gpcp3_ocean']=(('years'),total_list_ocean)
dk['freq_list_gpcp3_ocean']=(('years'),freq_list_ocean)
dk['intensity_list_gpcp3_ocean']=(('years'),intensity_list_ocean)

dk['total_list_gpcp3_land']=(('years'),total_list_land)
dk['freq_list_gpcp3_land']=(('years'),freq_list_land)
dk['intensity_list_gpcp3_land']=(('years'),intensity_list_land)

dk['total_gpcp3_ocean_monthly']=(('months','years'),total_ocean_monthly)
dk['freq_gpcp3_ocean_monthly']=(('months','years'),freq_ocean_monthly)
dk['intensity_gpcp3_ocean_monthly']=(('months','years'),intensity_ocean_monthly)

dk['total_gpcp3_land_monthly']=(('months','years'),total_land_monthly)
dk['freq_gpcp3_land_monthly']=(('months','years'),freq_land_monthly)
dk['intensity_gpcp3_land_monthly']=(('months','years'),intensity_land_monthly)

dk.coords['years']=np.arange(2001,2021,1)
dk.coords['months']=np.arange(12)

try:
    os.remove(dir_data+'precip_datasets_compare_gpcp3_wet_6070_test.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'precip_datasets_compare_gpcp3_wet_6070_test.nc',mode='w',format='NETCDF4')